In [33]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [34]:
# Import packages
from pyspark.sql import SparkSession
import time
import pandas as pd

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [35]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df = pd.read_csv(url)


In [36]:
# 2. Create a temporary view of the DataFrame.
spark_df = spark.createDataFrame(df)
spark_df.createOrReplaceTempView("home_sales")
spark_df.show(20)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [37]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
results = spark.sql(
    "SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price FROM home_sales WHERE bedrooms = 4 GROUP BY year(date) ORDER BY YEAR(date)")
results.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [38]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
results_2 = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price 
    FROM home_sales 
    WHERE bedrooms = 3 AND bathrooms = 3 
    GROUP BY YEAR(date) 
    ORDER BY YEAR(date)
""")

results_2.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [39]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
results_3 = spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND SQFT_living >= 2000
GROUP BY YEAR(date)
ORDER BY YEAR(date)
""")

results_3.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019|289859.14|
|2020|292289.09|
|2021|296330.96|
|2022|290242.99|
+----+---------+



In [40]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result_4 = spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_sales
WHERE price >= 350000
""")

result_4.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.12193894386291504 seconds ---


In [41]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache().createOrReplaceTempView("home_cache")

In [42]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [43]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

result_5 = spark.sql("""
SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM home_cache
WHERE price >= 350000
""")

result_5.show()


print("--- %s seconds ---" % (time.time() - start_time))

print("--- %s seconds ---" % (time.time() - start_time))


+---------+--------+
|avg_price|avg_view|
+---------+--------+
|473796.26|  32.264|
+---------+--------+

--- 0.18905282020568848 seconds ---
--- 0.18907499313354492 seconds ---


In [44]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import pandas as pd
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
df = pd.read_csv(url)
spark_df = spark.createDataFrame(df)
spark_df.createOrReplaceTempView("home_sales")
spark.catalog.cacheTable("home_sales")
home_cache = spark.table("home_sales")
home_cache.write.partitionBy("date_built").mode("overwrite").parquet("home_partitioned")

In [45]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("home_partitioned")

In [46]:
# 12. Create a temporary table for the parquet data.
parquet_data = spark.read.parquet("home_partitioned")


In [47]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

result_6 = spark.sql("""
    SELECT ROUND(AVG(price), 2) AS avg_price, AVG(view) AS avg_view
FROM parquet_sales
WHERE price >= 350000
""")

result_6.show()

print("--- %s seconds ---" % (time.time() - start_time))

24/08/10 20:25:23 ERROR Executor: Exception in task 2.0 in stage 51.0 (TID 240)
org.apache.spark.SparkFileNotFoundException: File file:/Users/stephenmarkovic/Downloads/Starter_Code 21/home_partitioned/date_built=2016/part-00009-fe3742fe-9599-4230-9abe-a61007b7d608.c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at org.apa

Py4JJavaError: An error occurred while calling o285.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 238) (stephens-mbp executor driver): org.apache.spark.SparkFileNotFoundException: File file:/Users/stephenmarkovic/Downloads/Starter_Code 21/home_partitioned/date_built=2016/part-00003-fe3742fe-9599-4230-9abe-a61007b7d608.c000.snappy.parquet does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execut

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")
